In [1]:
import json

data = []
with open("/Users/philip/Documents/PhD/data/ArenaData/arena_fits/2015-07-05.json") as f:
    data = f.readlines()

json_lines = []

for line in data:
    jsline = json.loads(line)
    json_lines.append(jsline)

In [2]:
import pandas as pd

frame = pd.DataFrame.from_dict(json_lines)

In [3]:
# rebuild dataframe
# make dataframe of dicts nested in 'value' column
value = pd.DataFrame(list(frame['value']))
del frame['value']

# make dataframe of dicts nested in 'trackeeHistory' column
trackee = pd.DataFrame(list(value['trackeeHistory']))
del value['trackeeHistory']

chi2PerDof = pd.DataFrame(list(trackee['chi2PerDof']))
chi2PerDof.columns = ['chi2PerDof']
probChi2 = pd.DataFrame(list(trackee['probChi2']))
probChi2.columns = ['probChi2']
nMeasurements = pd.DataFrame(list(trackee['nMeasurements']))
nMeasurements.columns = ['nMeasurements']

In [4]:
# make dataframe with a 'coordinates' column
averagecoordinate = pd.DataFrame(list(value['averagecoordinate']))
coordinates = pd.DataFrame(list(averagecoordinate['avg']))
averagecoordinate = averagecoordinate.join(coordinates)
error = pd.DataFrame(list(averagecoordinate['error']))
errorcoordinates = pd.DataFrame(list(error['coordinates']))
del errorcoordinates[2]
errorcoordinates.columns = ['x_error','y_error']

del averagecoordinate['avg']
del value['averagecoordinate']

# join dataframes
frame = frame.join(value.join(averagecoordinate))
frame = frame.join(chi2PerDof)
frame = frame.join(probChi2)
frame = frame.join(errorcoordinates)
frame = frame.join(nMeasurements)
del frame['regionsNodesIds']
del frame['error']
del frame['type']

In [6]:
frame = frame[(frame['probChi2'] > 0.3) & 
              (frame['x_error'] < 10) & 
              (frame['y_error'] < 10) & 
              (frame['chi2PerDof'] < 1.2)]

In [5]:
frame = frame.sort_values(by='measurementTimestamp')

In [6]:
# get most frequent mac address
macs = frame['sourceMac'].value_counts()
macs[:5]

2d1ff804-c499-4163-b90f-003df1e4ec65    78664
85a84e08-c967-41d0-b56b-522574796776    74356
2c019c2d-7e9e-42b1-b3bd-1797c918d4a7    16995
e1deda99-163f-4b68-9ade-e1f05d070bf2     8370
6cab229a-e15d-48b7-ab1e-5ddca3d7e283     7712
Name: sourceMac, dtype: int64

In [7]:
len(macs)

30344

In [39]:
from math import floor, ceil, sqrt, pi, exp
import numpy as np
import time

begin = time.time()

width = 80; height = 60; 
periods = 10; nSubwindows = 1

bins = np.zeros((periods, height, width))

interval = 120000/nSubwindows # milliseconds
shift = 30000

weights = [1.6, 1.2, 0.8, 0.4]

def kernel(u):
    return 1/sqrt(2*pi) * exp(-(u**2)/2)

for k in range(periods):
    for m in range(nSubwindows):
        #start = min(frame['measurementTimestamp']) + k * interval
        #stop = min(frame['measurementTimestamp']) + (k+1) * interval

        start = min(frame['measurementTimestamp']) + k * shift + m * interval
        stop = start + interval
        
        subwindow = frame[(frame['measurementTimestamp'] >= start) & 
                           (frame['measurementTimestamp'] < stop)]
        
        subBins = np.zeros((nSubwindows, height, width))

        for i in range(len(macs)): # len(macs)
            # filter for mac address
            mac_filtered_frame = subwindow[subwindow['sourceMac'] == macs.index[i]]
            # extract positions
            locations = mac_filtered_frame['coordinates']

            positions1 = pd.DataFrame(locations)
            positions = pd.DataFrame(list(positions1['coordinates']))
            
            x_errors = pd.DataFrame(mac_filtered_frame['x_error'])
            y_errors = pd.DataFrame(mac_filtered_frame['y_error']) 

            # binned positions, still fractional numbers
            binpos = positions.apply(lambda x: x/3)

            normalization = 0
            
            # 2d binned region for one mac address
            bins_mac = np.zeros((height,width))

            gx = np.zeros(width)
            gy = np.zeros(height)

            for j in range(len(binpos)):
                # bin positions
                xbin = width/2 + floor(binpos[0][j])
                ybin = height/2 + floor(binpos[1][j])
                
                if xbin >= 0 and xbin < width and ybin >= 0 and ybin < height:
                    
                    normalization += 1
                    '''
                    bins_mac_temp = np.zeros((height,width))
                    
                    bins_mac_temp[ybin][xbin] += 1
                
                    #### apply Gaussian blurring #########################################
                    
                    smooth_bins = np.zeros((height,width))
                    
                    sigmaX = (x_errors.values[j]/3)**2
                    sigmaY = (y_errors.values[j]/3)**2
                    
                    # Gaussian kernel functions
                    for binNr in range(height):
                        gy[binNr] = 1/sqrt(2*pi*sigmaY) * exp(-(binNr**2)/(2*sigmaY))
                        
                    for binNr in range(width):
                        gx[binNr] = 1/sqrt(2*pi*sigmaX) * exp(-(binNr**2)/(2*sigmaX))
                    
                    # convolutions
                    for u in range(width):
                        for v in range(width):
                            smooth_bins[ybin, u] += bins_mac_temp[ybin, v] * gx[abs(v - u)]
                            
                    bins_mac_temp = smooth_bins
                    smooth_bins = np.zeros((height,width))
                            
                    for w in range(width):
                        for u in range(height):
                            for v in range(height):
                                smooth_bins[u, w] += bins_mac_temp[v, w] * gy[abs(v - u)]
                                
                    ######################################################################
                    '''
                    #### kernel density estimation #########################################

                    hx = x_errors.values[j]
                    hy = y_errors.values[j]

                    smooth_bins = np.zeros((60,80))

                    for i in range(width):
                        for j in range(height):
                            smooth_bins[j][i] += kernel((i - xbin) / hx) * kernel((j - ybin) / hy)
                            
                    smooth_bins /= hx * hy
                    
                    ######################################################################
                    
                    bins_mac += smooth_bins
                    
                    #print(smooth_bins.sum())
            
            #bins_mac *= weights[m]
            
            if normalization > 0:
                bins_mac /= normalization
                
            subBins[m] += bins_mac
    bins[k] = subBins.sum(axis=0)
    
    #### write 2d bin to file ####
    
    np.savetxt('bins_%d.txt' %  k, bins[k], delimiter=',')
    
    print('Time window:', k)
    
end = time.time()
print('Time elpased:', end - begin)

Time window: 0
Time window: 1
Time window: 2
Time window: 3
Time window: 4
Time window: 5
Time window: 6
Time window: 7
Time window: 8
Time window: 9
Time elpased: 2668.9994370937347


In [32]:
bins[0].sum()

460.30112436332843

In [33]:
# this is a plot test cell

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

fig = plt.figure(figsize=(12,8))

X = np.arange(width)
Y = np.arange(height)*(-1)
Xs, Ys = np.meshgrid(X, Y)

Z = np.loadtxt('bins_%d.txt' % 0, delimiter=',')

ax = Axes3D(fig)
ax.plot_surface(Xs, Ys, Z, rstride=2, cstride=1, cmap='hsv')#, cmap='hot')
#ax.set_zlim3d(0, 0.05)
plt.show()

In [132]:
bins[0].sum()

15.545633083190562

In [133]:
bins.shape

(1, 60, 80)

In [41]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

fig = plt.figure(figsize=(12,8))

X = np.arange(width)
Y = np.arange(height)*(-1)
Xs, Ys = np.meshgrid(X, Y)

for i in range(periods):
    Z = np.loadtxt('bins_%d.txt' % i, delimiter=',')

    ax = Axes3D(fig)
    ax.plot_surface(Xs, Ys, Z, rstride=2, cstride=1, cmap='hsv')#, cmap='hot')
    ax.set_zlim3d(0, 4)
    if i < 10:
        number = '000' + str(i)
    elif i > 9:
        number = '00' + str(i)
    elif i > 99:
        number = '0' + str(i)
    plt.savefig('surface-%s.png' % number)
#plt.show()


In [67]:
import matplotlib.pyplot as plt

#im = plt.imread('/Users/philip/Documents/PhD/data-analysis/escience/arena_sensation.png')

#fig = plt.figure()
#ax = fig.add_subplot(1,1,1)
# convenience method:
fig, ax = plt.subplots(figsize=(12,8))

#ax.set_xlim((0,1853)) # 1853
#ax.set_ylim((0,1369)) # 1369

#ax.imshow(im)

for i in range(periods):
    ax.imshow(bins[i], aspect='auto', cmap='hsv', alpha=1)
    if i < 10:
        number = '000' + str(i)
    elif i > 9:
        number = '00' + str(i)
    elif i > 99:
        number = '0' + str(i)
    plt.savefig('map-%s.png' % number)
    
#plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111, projection='3d')

x_data, y_data = np.meshgrid( np.arange(width),
                              np.arange(height)*(-1) )

col = ['r', 'y', 'c', 'k', 'c','r'] * height * width
# colors = np.random.choice(col, height*width)

x_data = x_data.flatten()
y_data = y_data.flatten()

for i in range(periods):
    z_data = bins[i].flatten()
    ax.set_zlim3d(0, 16)
    ax.bar3d( x_data,
              y_data,
              np.zeros(len(z_data)),
              1, 1, z_data, color=col) # 
    if i < 10:
        number = '000' + str(i)
    elif i > 9:
        number = '00' + str(i)
    elif i > 99:
        number = '0' + str(i)
    plt.savefig('bar-%s.png' % number)

#plt.show()